In [1]:
#uploading the dataset
from google.colab import files
uploaded = files.upload()


ModuleNotFoundError: No module named 'google'

In [ ]:
#importing the dataset
import pandas as pd
import io
raw_dataset = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

In [ ]:
#drop unique observations feature URL
#and drop the state 
feature for being useless and full of missing data
raw_dataset = raw_dataset.drop(columns=['URL','WHOIS_STATEPRO'])

In [ ]:
#split the features and the output/y
#features are the subset than contains all rows and all columns excpet last column
features = raw_dataset.iloc[:, :-1].values
#y is the subset that contains all rows of the last column
y = raw_dataset.iloc[:, raw_dataset.shape[1] - 1].values

In [ ]:
#take care of missing data
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = 'None',strategy = 'most_frequent')
charsetImputer = imputer.fit(features[:, 2:3])
features[:, 2:3] = charsetImputer.transform(features[:, 2:3])

imputer = SimpleImputer(strategy = 'most_frequent')
serverImputer = imputer.fit(features[:, 3:4])
features[:, 3:4] = serverImputer.transform(features[:, 3:4])
imputer = SimpleImputer(missing_values = 'None',strategy = 'most_frequent')
serverImputer = imputer.fit(features[:, 3:4])
features[:, 3:4] = serverImputer.transform(features[:, 3:4])

imputer = SimpleImputer(strategy = 'mean')
contLengthImputer = imputer.fit(features[:, 4:5])
features[:, 4:5] = contLengthImputer.transform(features[:, 4:5])

imputer = SimpleImputer(missing_values = 'None',strategy = 'most_frequent')
countryImputer = imputer.fit(features[:, 5:6])
features[:, 5:6] = countryImputer.transform(features[:, 5:6])

imputer = SimpleImputer(strategy = 'mean')
restImputer = imputer.fit(features[:, 8:18])
features[:, 8:18] = restImputer.transform(features[:, 8:18])

In [ ]:
#Encoding categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
features[:, 2] = labelencoder.fit_transform(features[:, 2])
features[:, 3] = labelencoder.fit_transform(features[:, 3])
features[:, 5] = labelencoder.fit_transform(features[:, 5])

In [ ]:
#Convert regdate to a number with reference to the oldest date
import datetime

referenceDateStr = '01/01/1985 00:00'
referenceDate = datetime.datetime.strptime(referenceDateStr, '%d/%m/%Y %H:%M')

for i in range(features.shape[0]):
  try:
    thisDate = datetime.datetime.strptime(features[i, 6], '%d/%m/%Y %H:%M')
  except ValueError:
    features[i, 6] = 'None'
    continue
  diff = thisDate - referenceDate
  features[i, 6] = diff.days


In [ ]:
#Convert upDate to a number with reference to the oldest date
referenceDateStr = '01/01/1985 00:00'
referenceDate = datetime.datetime.strptime(referenceDateStr, '%d/%m/%Y %H:%M')

for i in range(features.shape[0]):
  try:
    thisDate = datetime.datetime.strptime(features[i, 7], '%d/%m/%Y %H:%M')
  except ValueError:
    features[i, 7] = 'None'
    continue
  diff = thisDate - referenceDate
  features[i, 7] = diff.days

In [ ]:
#now deal with the issue of wrong dates

imputer = SimpleImputer(missing_values = 'None',strategy = 'most_frequent')
regDateImputer = imputer.fit(features[:, 6:7])
features[:, 6:7] = regDateImputer.transform(features[:, 6:7])

imputer = SimpleImputer(missing_values = 'None',strategy = 'most_frequent')
upDateImputer = imputer.fit(features[:, 7:8])
features[:, 7:8] = upDateImputer.transform(features[:, 7:8])

In [ ]:
#split features and y into test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=42)

In [ ]:
#scale features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
#logistic regression classifier with all features
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
lr_clf.predict(X_train)
print('Training Accuracy = %.2f%%, Test Accuracy = %.2f%%'%(lr_clf.score(X_train, y_train)* 100, lr_clf.score(X_test, y_test)* 100))

Training Accuracy = 93.05%, Test Accuracy = 91.32%


In [ ]:
#logistic regression classifier with PCA used for dimensionality reduction
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

for cov in [0.9, 0.95, 0.99]:
  print('\n')
  pca = PCA(n_components = cov)
  X_trainPca = pca.fit_transform(X_train)
  X_testPca = pca.fit_transform(X_test)
  expCov = sum(pca.explained_variance_ratio_)
  print("Number of dimensions when conserving %.0f%% of variance = %d"%(expCov*100, X_trainPca.shape[1]))
  lr_clfPca = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_trainPca, y_train)
  lr_clfPca.predict(X_trainPca)
  print('Training Accuracy = %.2f%%, Test Accuracy = %.2f%%'%(lr_clfPca.score(X_trainPca, y_train)* 100, lr_clfPca.score(X_testPca, y_test)* 100))
  print('-'*100)



Number of dimensions when conserving 93% of variance = 9
Training Accuracy = 88.69%, Test Accuracy = 57.98%
----------------------------------------------------------------------------------------------------


Number of dimensions when conserving 96% of variance = 10
Training Accuracy = 88.69%, Test Accuracy = 58.26%
----------------------------------------------------------------------------------------------------


Number of dimensions when conserving 99% of variance = 12
Training Accuracy = 91.71%, Test Accuracy = 85.15%
----------------------------------------------------------------------------------------------------


In [ ]:
#Logistic Regression with date columns dropped
from sklearn.linear_model import LogisticRegression
import numpy as np


X_train_no_date = np.delete(X_train, np.s_[7:9], 1)
X_test_no_date = np.delete(X_test, np.s_[7:9], 1)
lr_clf_no_date = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train_no_date, y_train)
lr_clf_no_date.predict(X_train_no_date)
print('Training Accuracy = %.2f%%, Test Accuracy = %.2f%%'%(lr_clf_no_date.score(X_train_no_date, y_train)* 100, lr_clf_no_date.score(X_test_no_date, y_test)* 100))

Training Accuracy = 92.91%, Test Accuracy = 91.32%


In [ ]:
#Logistic Regression with date & country columns dropped
from sklearn.linear_model import LogisticRegression

X_train_no_count = np.delete(X_train, np.s_[6:9], 1)
X_test_no_count = np.delete(X_test, np.s_[6:9], 1)
lr_clf_no_count = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train_no_count, y_train)
lr_clf_no_count.predict(X_train_no_count)
print('Training Accuracy = %.2f%%, Test Accuracy = %.2f%%'%(lr_clf_no_count.score(X_train_no_count, y_train)* 100, lr_clf_no_count.score(X_test_no_count, y_test)* 100))

Training Accuracy = 92.56%, Test Accuracy = 91.88%


In [ ]:
#Logistic Regression with Charset & server columns dropped
from sklearn.linear_model import LogisticRegression

X_train_no_count = np.delete(X_train, np.s_[2:4], 1)
X_train_no_count = np.delete(X_train_no_count, np.s_[3:6], 1)
X_test_no_count = np.delete(X_test, np.s_[2:4], 1)
X_test_no_count = np.delete(X_test_no_count, np.s_[3:6], 1)
lr_clf_no_count = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train_no_count, y_train)
lr_clf_no_count.predict(X_train_no_count)
print('Training Accuracy = %.2f%%, Test Accuracy = %.2f%%'%(lr_clf_no_count.score(X_train_no_count, y_train)* 100, lr_clf_no_count.score(X_test_no_count, y_test)* 100))

Training Accuracy = 90.31%, Test Accuracy = 89.36%


In [ ]:
#Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
gnb_model = GaussianNB() 
gnb_model.fit(X_train, y_train) 
  
# making predictions on the testing set 
y_pred = gnb_model.predict(X_test) 
y_train_pred = gnb_model.predict(X_train)
train_accuracy = gnb_model.score(X_train,y_train)
test_accuracy = gnb_model.score(X_test,y_test)
print(f"Training Accuracy: {train_accuracy*100:.2f}, Testing Accuracy: {test_accuracy*100:.2f}")  

## Cross Validation
results =  cross_validate(gnb_model, X_train, y_train, cv= 5,return_train_score=True,scoring='neg_mean_squared_error')
test_score =  -np.mean(results['test_score'])
training_score = -np.mean(results['train_score'])
print("trainig cross validation accuracy = %f , cross valididation test accuracy = %f"%(1-training_score,1-test_score))

# comparing actual response values (y_test) with predicted response values (y_pred) 
#from sklearn import metrics 
#print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)
#print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_train, y_train_pred)*100)

Training Accuracy: 39.40, Testing Accuracy: 82.63
trainig cross validation accuracy = 0.466417 , cross valididation test accuracy = 0.457388


In [ ]:
# Random Forest Classifier using Information Gain criterion, maximum depth of 9, 5 classifiers, and random_state=3
from sklearn.ensemble import RandomForestClassifier

classifier= RandomForestClassifier(max_depth=9, n_estimators=5, criterion= 'gini')
classifier = classifier.fit(X_train,y_train)
train_results = classifier.score(X_train,y_train)
test_results =classifier.score(X_test ,y_test)

print('training accuracy = %f,  testing accuracy = %f'%(train_results, test_results))
# using cross validation to decrease over fitting
results =  cross_validate(classifier, X_train, y_train, cv= 5,return_train_score=True,scoring='neg_mean_squared_error')
test_score =  -np.mean(results['test_score'])
training_score = -np.mean(results['train_score'])
print("trainig cross validation accuracy = %f , cross valididation test accuracy = %f"%(1-training_score,1-test_score))


training accuracy = 0.984551,  testing accuracy = 0.851541
trainig cross validation accuracy = 0.990696 , cross valididation test accuracy = 0.955755


In [ ]:
#  Adaboost Ensemble Model using decision trees classifier with Information Gain criterion, maximum depth of 3, 10 classifiers, and random_state=3
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

d_tree= DecisionTreeClassifier(max_depth=2)
classifier= AdaBoostClassifier(base_estimator=d_tree, n_estimators=17)

classifier = classifier.fit(X_train,y_train)
train_results = classifier.score(X_train,y_train)
test_results =classifier.score(X_test,y_test)

print('training accuracy = %f,  testing accuracy = %f'%(train_results, test_results))

#using Cross validation to decrease overfitting
results =  cross_validate(classifier, X_train, y_train, cv= 5,return_train_score=True,scoring='neg_mean_squared_error')
test_score =  -np.mean(results['test_score'])
training_score = -np.mean(results['train_score'])
print("trainig cross validation accuracy = %f , cross valididation test accuracy = %f"%(1-training_score,1-test_score))


training accuracy = 0.990871,  testing accuracy = 0.879552
trainig cross validation accuracy = 0.994733 , cross valididation test accuracy = 0.962780


In [ ]:
from sklearn.svm import SVC # "Support Vector Classifier"
from sklearn.model_selection import cross_validate
import numpy as np

classifier = SVC(kernel='linear') 
classifier = classifier.fit(X_train,y_train)
train_results = classifier.score(X_train,y_train)
test_results =classifier.score(X_test,y_test)
print('training accuracy = %f,  testing accuracy = %f'%(train_results, test_results))
results =  cross_validate(classifier, X_train, y_train, cv= 5,return_train_score=True,scoring='neg_mean_squared_error')
test_score =  -np.mean(results['test_score'])
training_score = -np.mean(results['train_score'])
print("trainig cross validation accuracy = %f , cross valididation test accuracy = %f"%(1-training_score,1-test_score))


training accuracy = 0.936798,  testing accuracy = 0.910364
trainig cross validation accuracy = 0.937675 , cross valididation test accuracy = 0.935389


In [ ]:
from sklearn.svm import SVC # "Support Vector Classifier"
from sklearn.model_selection import cross_validate
import numpy as np

classifier = SVC(gamma='scale') 
classifier = classifier.fit(X_train,y_train)
train_results = classifier.score(X_train,y_train)
test_results =classifier.score(X_test,y_test)
print('training accuracy = %f,  testing accuracy = %f'%(train_results, test_results))
results =  cross_validate(classifier, X_train, y_train, cv= 5,return_train_score=True,scoring='neg_mean_squared_error')
test_score =  -np.mean(results['test_score'])
training_score = -np.mean(results['train_score'])
print("trainig cross validation accuracy = %f , cross valididation test accuracy = %f"%(1-training_score,1-test_score))

training accuracy = 0.945927,  testing accuracy = 0.927171
trainig cross validation accuracy = 0.944522 , cross valididation test accuracy = 0.940309
